# Working with OpenAI's API

## Introduction

In this notebook, we'll go over working with OpenAI's API through [their python client](https://github.com/openai/openai-python). Specifically, we'll:

- Setup our OpenAI API key.
- Explore the API.
- Use the API to generate text.
- Run the output through OpenAI's moderation endpoint.


## Setup

Let's get started by importing our libraries.

In [18]:
import openai
import configparser

If you haven't already done so, go ahead and create a `.env` file in the same directory as this notebook (notebooks) and fill it out like this:

```
[openai]
OPENAI_API_KEY = sk-*******
```

You can find your API key [here](https://platform.openai.com/account/api-keys) after your log in. Replace what we have above with your key. Do not use quotes around your key.

Once you've done this, we can load our key into this notebook.

In [19]:
# Parse config
cp = configparser.ConfigParser(interpolation=None)
cp.read("./.env")

# Let's load the Hearst credentials
openai.api_key = cp.get("openai", "OPENAI_API_KEY")

## Explore the API

Now that we've loaded our key, let's explore the API.

We'll get started by creating some variables that will control how the LLM behaves.

First we'll define which model we want to use. We'll use GPT-3.5-turbo. For more sophisticated tasks, you may want to use GPT-4. For a reminder of all of the models available to you and the differences among them, please refer to [OpenAI's documentation](https://platform.openai.com/docs/models/gpt-3-5).

Then we'll define the temperature. This setting controls how "creative" the model is. A higher temperature will result in more creative output, but it will also be less accurate. A lower temperature will result in more accurate output, but it will also be less creative. The range is 0 to 2. Never go above 1.

Finally, we'll define the max_tokens. This tells the LLM the max number of [tokens](https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them) the model should generate. Remember, OpenAI charges by the token for usage of their API. Inputs are counted toward the cost. 

In [20]:
MODEL = "gpt-3.5-turbo"
TEMPERATURE = 0
MAX_TOKENS = 100

Now let's create our system and user prompts.

System prompts tells the llm how to behave. This is something that is defined behind the scenes when you work with chatGPT's app interface. It is not required.

User prompts are the prompts that you provide to the llm. These are the prompts that will be used to generate the output. This is what you're used to see with chatGPT's app interface.

For this example, we'll have the LLM summarize a news article.

In [21]:
SYSTEM_PROMPT = "You are a helpful virtual assistant."

USER_PROMPT = """Pretend to be a reporter at the Washington Post. Below I'm providing you a snippet of a news article that is wrapped in triple backticks (```). Summarize the content using easy-to-understand language. Summaries must be one paragraph. Do not make any assumptions. Only pull information from the snippet.

```
San Antonio will begin offering another nonstop flight in June, this time to Torreón, Mexico — a hub of the electronics and automotive industries.

Wednesday's announcement of the new route on Viva Aerobus came a week after the Mexican low-cost carrier said it would begin offering nonstop service in December to Querétaro, Mexico, which also has a strong automotive industry. 

Both destinations are of potential interest to those involved in the San Antonio region’s growing auto industry and others. 

Airport Director Jesus Saenz said pursuing a route to Torreón in northern Mexico was “a strategic decision” and that the service will benefit both business and leisure travelers. Travel time will be as much as six hours faster with the nonstop flight than is now available from San Antonio.
```

Summary: """

OK, let's bring it all together into a call to the API.

In [23]:
response = openai.ChatCompletion.create(
    model=MODEL,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT}
    ]
)

response

<OpenAIObject chat.completion id=chatcmpl-89EnrBR1fMrnfLyRCG0pD0wRbqbWD at 0x118743c50> JSON: {
  "id": "chatcmpl-89EnrBR1fMrnfLyRCG0pD0wRbqbWD",
  "object": "chat.completion",
  "created": 1697212547,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "San Antonio will be adding another nonstop flight to its list of destinations, this time to Torre\u00f3n, Mexico. The new route, offered by Viva Aerobus, is seen as a strategic decision that will benefit both business and leisure travelers. Torre\u00f3n is known for its strong presence in the electronics and automotive industries, making it of potential interest to those involved in San Antonio's growing auto industry. The nonstop flight will also significantly reduce travel time, making it a more convenient option"
      },
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 248,
    "completion_tokens": 100,
    "total_toke

Let's break down what you're seeing above.

You're receiving a JSON object back from the API. The object contains the following:

- **model:** The model used to generate the output. As of this writing, it says "gpt-3.5-turbo-0613". That's different from what we defined! Here's the deal: OpenAI tweaks their models over time. When you just define "gpt-3.5-turbo", you're telling the API to use the latest version of that model. If you want to use a specific version, you can go to the model listing and select which version you want from there.
- **choices:** This is the output from the LLM. It's an array of objects. It includes our assistant's message back to us in response to our prompts. You may notice that it finished with an incomplete sentence. This reason for this is because we told the LLM "hey, only use up to 100 tokens." The LLM is incapable of thinking ahead of time that it only has 100 tokens to work with, so it'll just end the sentence when it hits that limit.
- **usage:** This is telling us how many tokens we used. Remember, OpenAI charges by the token. Inputs are counted toward the cost.

If we only want the output, we can do this:

In [26]:
output = response["choices"][0]["message"]["content"]

print(output)